## Cleaning factRuEval-2016

This dataset is somewhat challenging. The tokens, spans, objects (labels), and text are all being kept in separate files...for every single training/test observation. 

To summarize the work that is being done below:

The goal is to get the data into a format where we have tokens on the left and their labels on the right in an easily readable text file. To do this, we need to join the token file to the span file to the object file (because there is insufficient information in the token file to join it to the object file directly).

Steps:

1. Read in the token data.
2. Read in and process the span data.
3. Process the label data, removing duplicates / "recursive" labels.
4. Combine the tokens, spans, and labels.
5. Filter out unwanted label types ("fact" labels vs. NER labels, because fact-finding was a separate task for this dataset)
6. **(To do next)** Use the B-PER and I-PER convention to identify which tokens belong to the same label (for example, "National Bank of Ukraine").

The way a couple of these steps are implemented is likely not ideal for large datasets, but will probably work for now.

In [53]:
import pandas as pd
token_col_names = ['id', 'begin_index', 'length', 'text']
test_obj = pd.read_csv('../data/ru/factRuEval-2016/devset/book_194.tokens', 
                       sep=' ', skip_blank_lines=False, header=None)
test_obj.columns = token_col_names

test_obj[:15]

,id,begin_index,length,text
0,210354.0,0.0,6.0,Россия
1,210355.0,7.0,4.0,дала
2,210356.0,12.0,7.0,Украине
3,210357.0,20.0,6.0,кредит
4,210358.0,27.0,2.0,на
5,210359.0,30.0,5.0,сумму
6,210360.0,36.0,1.0,2
7,210361.0,38.0,9.0,миллиарда
8,210362.0,48.0,8.0,долларов
9,210363.0,56.0,1.0,","


In [201]:
test_file = '../data/ru/factRuEval-2016/devset/book_194.spans'

def extract_span_info(filename):
    
    with open(filename) as s:
        info = s.readlines()
    
    intermediate = [l.split() for l in info]
    
    # Extract only the necessary fields for each intermediate representation
    # Grab token ids + phrase text, span id, mention type, position, and length of span/phrase
    fields = [(i_rep[7:], i_rep[0], i_rep[1], i_rep[2], i_rep[5]) 
              for i_rep in intermediate]
    
    return fields

extracted = extract_span_info(test_file)

In [202]:
def separate_ids_and_text(id_text_list):
    
    if len(id_text_list) == 2:
        return [{id_text_list[0]: id_text_list[1]}]
    
    else:
        list_len = len(id_text_list)
        text_idx = list_len // 2
        return [{item[0]: item[1]} for item 
                in zip(id_text_list[:text_idx], id_text_list[text_idx:])
               ]
    
df = pd.DataFrame(extracted)
df[0] = df[0].apply(separate_ids_and_text)
df = df.explode(0)
df['token_id'] = df[0].apply(lambda x: list(x.keys())[0])
df['text'] = df[0].apply(lambda x: list(x.values())[0])
df.columns = ['0', 'span_id', 'mention_type', 'begin_index', 'span_length', 'token_id', 'text']
df = df[['token_id', 'span_id', 'text', 'span_length', 'begin_index', 'mention_type']]

# Convert dtypes for joining with token df
for col in ['token_id', 'span_length', 'begin_index']:
    df[col] = df[col].astype('int')

In [300]:
df[:10]

,token_id,span_id,text,span_length,begin_index,mention_type,label
0,210354,26213,Россия,1,0,loc_name,LocOrg
1,210356,26214,Украине,1,12,loc_name,LocOrg
2,210365,26215,Лента.ру,1,67,org_name,Org
3,210372,26216,Национальный,2,124,org_descr,Org
3,210373,26216,банк,2,124,org_descr,Org
4,210374,26217,Украины,1,142,loc_name,Org
5,210373,26218,банк,1,137,org_descr,Org
6,210370,38269,правительство,1,104,org_descr,Org
7,210369,38270,украинское,2,93,org_descr,Org
7,210370,38270,правительство,2,93,org_descr,Org


In [283]:
# Read in and process the labels
test_labels = '../data/ru/factRuEval-2016/devset/book_194.objects'

# This is NER, so each label can have one or more token IDs associated with it.
# Each line/list `l` holds the token IDs from index 2 up to the hash symbol.
with open(test_labels) as t:
    labs = [l.split() for l in t.readlines()]
    ids_labs_only = [(l[2: l.index('#')], l[1]) for l in labs]
 
# `seen` keeps track of span_ids we have already added as key-label pairs.
# Some label files have redundant tags (spans have multiple labels),
# so we need to only include the first instance of each span.
# Open book_194.objects for an example of this; span 26217, 'Украины',
# is tagged both as part of an Org AND on its own as LocOrg, even though
# it only occurs once in the sentence.
ind_dicts = []
seen = [] 
for item in ids_labs_only:
    for span_id in item[0]:
        if span_id not in seen:
            ind_dicts.append({span_id: item[1]})
            seen.append(span_id)
        
# Update the first dict with all the other k-v pairs 
for d in ind_dicts:
    ind_dicts[0].update(d)
    
# `label_dict` is now a dict with span_ids as keys and their labels as values.
label_dict = ind_dicts[0]

get_label = lambda x: label_dict.get(x, np.nan)
df['label'] = df['span_id'].apply(get_label)

In [298]:
labeled_tokens_df = test_obj.merge(df, 
                                    left_on=['id', 'begin_index', 'text'], 
                                    right_on=['token_id', 'begin_index', 'text'], 
                                    how='left')

labeled_tokens_df[:20]

# Note to self: span_id may more accurately be called position_id. Rename later

,id,begin_index,length,text,token_id,span_id,span_length,mention_type,label
0,210354.0,0.0,6.0,Россия,210354.0,26213,1.0,loc_name,LocOrg
1,210355.0,7.0,4.0,дала,NaN,NaN,NaN,NaN,NaN
2,210356.0,12.0,7.0,Украине,210356.0,26214,1.0,loc_name,LocOrg
3,210357.0,20.0,6.0,кредит,NaN,NaN,NaN,NaN,NaN
4,210358.0,27.0,2.0,на,NaN,NaN,NaN,NaN,NaN
5,210359.0,30.0,5.0,сумму,NaN,NaN,NaN,NaN,NaN
6,210360.0,36.0,1.0,2,NaN,NaN,NaN,NaN,NaN
7,210361.0,38.0,9.0,миллиарда,NaN,NaN,NaN,NaN,NaN
8,210362.0,48.0,8.0,долларов,NaN,NaN,NaN,NaN,NaN
9,210363.0,56.0,1.0,",",NaN,NaN,NaN,NaN,NaN


In [301]:
# `cond` checks if the join was successful, but the text is unlabeled.
# (that is, "token_id NOT NULL, label IS NULL")
# If this is the case, we know it's a duplicate of a previous token,
# and was duplicated in the join with the span + label data (due to the 
# "redundant tags" issue mentioned above), so we can safely drop these.
cond = ~labeled_tokens_df['token_id'].isna() & labeled_tokens_df['label'].isna() 

# Invert the condition to get only the non-duplicate tokens
labeled_tokens_df = labeled_tokens_df[~cond]

# Now we're quite close to having a df of text and labels!
# Next step is to detect if the span length is > 1 and appropriately
# indicate in the label which tokens belong to the same "label"--
# for example "National Bank of Ukraine" should be labeled as a single
# entity. We can use I-PER + B-PER / I-ORG / B-ORG for this, as is 
# usual for NER datasets.
labeled_tokens_df[:13]

,id,begin_index,length,text,token_id,span_id,span_length,mention_type,label
0,210354.0,0.0,6.0,Россия,210354.0,26213,1.0,loc_name,LocOrg
1,210355.0,7.0,4.0,дала,NaN,NaN,NaN,NaN,NaN
2,210356.0,12.0,7.0,Украине,210356.0,26214,1.0,loc_name,LocOrg
3,210357.0,20.0,6.0,кредит,NaN,NaN,NaN,NaN,NaN
4,210358.0,27.0,2.0,на,NaN,NaN,NaN,NaN,NaN
5,210359.0,30.0,5.0,сумму,NaN,NaN,NaN,NaN,NaN
6,210360.0,36.0,1.0,2,NaN,NaN,NaN,NaN,NaN
7,210361.0,38.0,9.0,миллиарда,NaN,NaN,NaN,NaN,NaN
8,210362.0,48.0,8.0,долларов,NaN,NaN,NaN,NaN,NaN
9,210363.0,56.0,1.0,",",NaN,NaN,NaN,NaN,NaN
